In [1]:
import boto3
import pandas as pd
import numpy as np
import sys
from io import StringIO
import surprise
from surprise import accuracy
from sklearn.model_selection import train_test_split

In [2]:
sys.path.insert(1, '../src')
from Collaborative_recommender import CollaborativeRecommender

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%reload_ext autoreload

In [5]:
s3 = boto3.client('s3')

In [6]:
df_all_15 = pd.read_csv("s3://recommender-system-amazon/df_all_15_text.csv")

In [7]:
reviewerID = pd.read_csv("s3://recommender-system-amazon/reviewerID.csv")

In [8]:
reviewerID.drop(['Unnamed: 0',"asin"],axis=1,inplace=True)

In [9]:
df_all_15 = pd.concat([df_all_15, reviewerID.reindex(df_all_15.index)], axis=1)

In [10]:
df_all_15.head(2)

,Unnamed: 0,category,description,title,also_buy,image_x,brand,feature,rank,also_view,...,asin,overall,verified,reviewText,summary,unixReviewTime,style,item_text,title_nlp,reviewerID
0,0,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",...,0899332757,5.0,False,your interest explor area new york beyond metr...,Great for exploring the rural parts of the state,1470009600,{'Format:': ' Paperback'},garmingreat explor rural part stateformat pape...,delorm new york state atla amp gazett,A1LACH6MLQWZ
1,1,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",...,0899332757,4.0,True,wonder comprehens atla littl larg,Four Stars,1438646400,{'Format:': ' Paperback'},garminfour starsformat paperback,delorm new york state atla amp gazett,A2LBF5LI09BS8Z


In [13]:
df_all_15.shape

(369155, 22)

In [14]:
df_all_15 = df_all_15[~(df_all_15['title_nlp'].isna())]
df_all_15 = df_all_15[~(df_all_15['item_text'].isna())]
df_all_15 = df_all_15[~(df_all_15['reviewText'].isna())]

In [15]:
df_all_15.columns

Index(['Unnamed: 0', 'category', 'description', 'title', 'also_buy', 'image_x',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'price', 'asin',
       'overall', 'verified', 'reviewText', 'summary', 'unixReviewTime',
       'style', 'item_text', 'title_nlp', 'reviewerID'],
      dtype='object')

In [16]:
df_collab = df_all_15[['reviewerID',"asin","overall"]]

In [17]:
df_collab.head()

,reviewerID,asin,overall
0,A1LACH6MLQWZ,0899332757,5.0
1,A2LBF5LI09BS8Z,0899332757,4.0
2,A11G1FW7QISV3Y,0899332757,5.0
3,APR0HI8KAJ4QG,0899333257,5.0
4,A1LACH6MLQWZ,0899333257,5.0


In [66]:
df_collab.shape

(369155, 3)

In [68]:
UM = pd.pivot_table(df_collab,index='reviewerID', columns='asin', values='overall')

In [71]:
UM

asin,0899332757,0899333257,0971100764,3843518912,7245456275,7245456313,7245456453,8804850086,9641766805,B00000IURU,...,B01HJ234E0,B01HJ4DN08,B01HJ4EYS8,B01HJ8YC0Y,B01HJA241G,B01HJA7KWE,B01HJDGJ1E,B01HJDHNX2,B01HJDZ34I,B01HJGAJ9O
reviewerID,,,,,,,,,,,,,,,,,,,,,
A0098869QS7I6WOAODWI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0418008MEWDDIUPHXP7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0743345UFTOA4V1Z7W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0784213CF0H1GDXG0G2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A07936821FOVJO6NP4Q8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZYJE40XW6MFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZYJMUE8PNTQV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZYP4FQ2L2C4O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
df_collab.describe()

,overall
count,369155.000000
mean,4.447806
std,0.965468
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [18]:
reader = surprise.Reader(rating_scale=(1,5))
# recom = CollaborativeRecommender(df_collab,reader)

In [38]:
# param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005]}
# recom.grid(param_grid,cv=3)
# recom.fit()
# recom.predict()

RMSE: 0.7925


0.7924577576585875

In [15]:
# X_Train,X_holdout = train_test_split(df_collab, test_size=0.2)

In [19]:
reader = surprise.Reader(rating_scale=(1,5))

In [20]:
# data = surprise.Dataset.load_from_df(X_Train,reader)

data = surprise.Dataset.load_from_df(df_collab,reader)
# holdout = surprise.Dataset.load_from_df(X_holdout,reader)

In [16]:
kSplit = surprise.model_selection.split.KFold(n_splits=1, shuffle=True)

In [75]:
sim_options = sim_options = {'name': 'cosine','user_based': False}

collabKNN = surprise.KNNBasic(k=100,min_k=1,sim_options=sim_options,verbose=True) #try removing sim_options. 
# collabKNN = surprise.SVD()
trainset = data.build_full_trainset()
collabKNN.fit(trainset)
testset = trainset.build_testset()
predictions = collabKNN.test(testset)
accuracy.rmse(predictions, verbose=True)

# rmseKNN = []



# for trainset, testset in kSplit.split(data): #iterate through the folds.
#     collabKNN.fit(trainset)
#     predictionsKNN = collabKNN.test(testset)
#     rmseKNN.append(surprise.accuracy.rmse(predictionsKNN,verbose=True))#get root means squared 

In [18]:
sm = collabKNN.compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [19]:
sm.shape

(56351, 56351)